## Variable scope

This set of notes is about building a program from scratch and learning about scope, one of the most confusing parts of programming for beginners. The "scope" is everything the computer can "see" at a particular point in the code, i.e. the names of variables and functions the computer has learnt (so far). We have to learn about scope, as many errors arise from not understanding it, and you want to keep your scope as clean as possible so that errors are easy to find.

To motivate it, we are going to build a prime number calculator!

Prime numbers are natural numbers (positive integers) which have only themselves and 1 as factors. Thus 2 is a prime number and so is 3, but not 4 as $2\times2=4$. There are no simple patterns to the primes. In fact, as soon as a pattern appears it seems to be destroyed. For example, the first two primes appear to be increasing by 1, but this is immediately broken by the next prime being 5. We might then hypothesise the increase in the primes to be +1,+2,+3 but then the next prime (7) is actually a +2 again!

There are some general statements we can make about primes, such as the probability of a number being prime is proportional to $1/\ln(n)$, but there is no fast way to test if a number is prime. One simple strategy is [trial division](https://en.wikipedia.org/wiki/Trial_division), where given a candidate prime number $n$, we test every integer less than $n$ to see if it divides $n$ an integer number of times.

Lets try $n=5$ by hand,

In [2]:
print(5/1)
print(5/2)
print(5/3)
print(5/4)
print(5/5)

5.0
2.5
1.6666666666666667
1.25
1.0


As it is only divisible by itself and 1, 5 is indeed prime. Lets make this test smarter and get the computer to do the work for us, we want to test all numbers from $[1,\,n]$, so lets generate a list of numbers,

In [7]:
list(range(1,6))

[1, 2, 3, 4, 5]

We need to go to `6` as range misses out the last number (for good reason, it normally starts at zero, so `range(5)` gives you 5 numbers from $[0,4]$. 

Using this we can get a loop to test all the divisors for us:

In [5]:
for divisor in range(1,5):
    print(5/divisor)

5.0
2.5
1.6666666666666667
1.25


Several things happened here. We dropped the `list()` around the range, as its more efficient (see previous slides about generators). We added a `for` loop to go through the list and create a variable `divisor` which points to each member of the list in turn.

But let's talk about scope. After the loop, does `divisor` still exist? Can we still use it? 

In [6]:
divisor

4

In [8]:
Yes!

SyntaxError: invalid syntax (<ipython-input-8-6bfbace0806d>, line 1)